In [1]:
import pandas as pd
import numpy as np
import warnings
import boto3
import re
from tqdm import tqdm
warnings.filterwarnings("ignore")

### data

In [2]:
client = boto3.client('s3')

In [3]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [4]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/latest_data/feature_prep/')

In [5]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    if ('base' in k or 'diff' in k) and ('rsi' not in k and 'macd' not in k):
        paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [6]:
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10D_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_10min_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_120min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_120min_diff.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_150D_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_b

In [7]:
def calculate_macd(df, signal=14, ema_columns=[]):
    for i in range(len(ema_columns)):
        for j in range(i + 1, len(ema_columns)):
            fast_ema = ema_columns[i]
            slow_ema = ema_columns[j]

            # Calculate MACD
            # macd_col_name = f'MACD_{fast_ema}_{slow_ema}'
            # df[macd_col_name] = df[fast_ema] - df[slow_ema]

            # Calculate Signal line
            signal_col_name = f'Signal_{fast_ema}_{slow_ema}_signal{signal}'
            # df[signal_col_name] = df[macd_col_name].ewm(span=signal, adjust=False).mean()
            df[signal_col_name] = (df[fast_ema] - df[slow_ema]).ewm(span=signal, adjust=False).mean()
            

            # # Calculate Histogram
            # histogram_col_name = f'Histogram_{fast_ema}_{slow_ema}_signal{signal}'
            # df[histogram_col_name] = df[macd_col_name] - df[signal_col_name]
    return df


def read_and_calculate_macd(path, signals):
    print(f'Reading from {path}')
    df = pd.read_parquet(path)
    fields = [z for z in df.columns if 'close_ema' in z or 'close_diff_ema' in z]
    df = df[['symbol'] + fields]
    grouped = df.groupby('symbol')
    results = []

    for symbol, group in grouped:
        ema_columns = [z for z in group.columns if z!='symbol']
        ema_columns = sorted(ema_columns, key=lambda x: int(re.search(r'\d+', x).group()))
        print(f'symbol - {symbol} for signals = {signals}')
        for signal in signals:
            # print(f'for signal = {signal}')
            group = calculate_macd(group, signal, ema_columns)
        results.append(group)

    # Concatenate all the grouped results back into a single DataFrame
    df = pd.concat(results)
    loc = path.replace('.parquet', '_macd.parquet')
    print(f'Saving to {loc}')
    df.to_parquet(loc)
    del df, group, results


for path in tqdm(paths):
    base_time_interval = path.split('/')[-1].split('.')[0].split('_')[-2]
    if 'min' in base_time_interval:
        if int(base_time_interval.replace('min', ''))<=10:
            signal = [11, 13, 17, 20, 26]
        elif int(base_time_interval.replace('min', ''))<=30:
            signal = [11, 13, 17, 20, 26]
        else:
            signal = [11, 13, 17, 20, 26, 30, 50]
    elif 'D' in base_time_interval:
        if int(base_time_interval.replace('D', '')) < 100:
            signal = [11, 13, 17, 20, 26]
        else: 
            signal = [11, 13, 17, 20, 26, 30, 50]
    read_and_calculate_macd(path, signal)

  0%|          | 0/50 [00:00<?, ?it/s]

Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base.parquet
symbol - QQQ for signals = [11, 13, 17, 20, 26, 30, 50]
symbol - SPY for signals = [11, 13, 17, 20, 26, 30, 50]
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base_macd.parquet


  2%|▏         | 1/50 [00:01<01:02,  1.27s/it]

Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff.parquet
symbol - QQQ for signals = [11, 13, 17, 20, 26, 30, 50]
symbol - SPY for signals = [11, 13, 17, 20, 26, 30, 50]
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff_macd.parquet


  4%|▍         | 2/50 [00:02<00:49,  1.02s/it]

...

 96%|█████████▌| 48/50 [02:57<00:09,  4.98s/it]

Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_base.parquet
symbol - QQQ for signals = [11, 13, 17, 20, 26, 30, 50]
symbol - SPY for signals = [11, 13, 17, 20, 26, 30, 50]
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_base_macd.parquet


 98%|█████████▊| 49/50 [02:59<00:04,  4.14s/it]

Reading from s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_diff.parquet
symbol - QQQ for signals = [11, 13, 17, 20, 26, 30, 50]
symbol - SPY for signals = [11, 13, 17, 20, 26, 30, 50]
Saving to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_diff_macd.parquet


100%|██████████| 50/50 [03:02<00:00,  3.64s/it]


### eda

In [8]:
# y = pd.read_parquet("s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base.parquet")
# y = pd.read_parquet("s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base.parquet")

In [9]:
x = pd.read_parquet("s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_diff_macd.parquet")
x.loc['2024-11-19 16:50:00']
print(x.shape) # (1160890, 149)
x = x.reset_index()[['us_eastern_timestamp', 'symbol']]

(429593, 149)


In [10]:
y = pd.read_parquet("s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_diff_macd.parquet")
y.loc['2024-11-19 16:50:00']
print(y.shape) # (392417, 149)
y = y.reset_index()[['us_eastern_timestamp', 'symbol']]

(214798, 149)


In [11]:
x[x.symbol=='SPY']

,us_eastern_timestamp,symbol
214799,2024-01-02 04:00:00,SPY
214800,2024-01-02 04:01:00,SPY
214801,2024-01-02 04:02:00,SPY
214802,2024-01-02 04:03:00,SPY
214803,2024-01-02 04:04:00,SPY
...,...,...
429588,2024-11-19 19:55:00,SPY
429589,2024-11-19 19:56:00,SPY
429590,2024-11-19 19:57:00,SPY
429591,2024-11-19 19:58:00,SPY


In [12]:
y[y.symbol=='SPY']

,us_eastern_timestamp,symbol
107401,2024-01-02 04:00:00,SPY
107402,2024-01-02 04:02:00,SPY
107403,2024-01-02 04:04:00,SPY
107404,2024-01-02 04:06:00,SPY
107405,2024-01-02 04:08:00,SPY
...,...,...
214793,2024-11-19 19:50:00,SPY
214794,2024-11-19 19:52:00,SPY
214795,2024-11-19 19:54:00,SPY
214796,2024-11-19 19:56:00,SPY


In [13]:
del x
# 175922 232320 

In [15]:
# y[(y.symbol=='SPY')&(~y.us_eastern_timestamp.isin(x[x.symbol=='SPY'].us_eastern_timestamp))]
# x[(x.symbol=='SPY')&(~x.us_eastern_timestamp.isin(y[y.symbol=='SPY'].us_eastern_timestamp))]

In [17]:
# y[y.symbol=='SPY'].tail(30)